<a href="https://colab.research.google.com/github/Anubhab0410/Generative-AI-Sketch-to-Real-Interior-Designer/blob/main/ControlNet_LoRA_FineTuning_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install compatible versions for Colab's Python 3.12 environment
!pip install -U diffusers transformers accelerate datasets
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# We skip xformers for now to avoid the schema mismatch error you encountered

In [ ]:
import torch
import os
from google.colab import drive
from datasets import load_dataset
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, UniPCMultistepScheduler

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Use your established path
DRIVE_DATA_DIR = "/content/drive/MyDrive/interior_design_dataset_A/content/interior_dataset"

# 3. Load Dataset
dataset = load_dataset("json", data_files=os.path.join(DRIVE_DATA_DIR, "train.jsonl"))

def map_paths(example):
    example["image"] = os.path.join(DRIVE_DATA_DIR, example["image"])
    example["conditioning_image"] = os.path.join(DRIVE_DATA_DIR, example["conditioning_image"])
    return example

train_dataset = dataset['train'].map(map_paths)
print(f"✅ Ready! Loaded {len(train_dataset)} pairs.")

In [ ]:
# 4. Load Models
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

# 5. Use Built-in PyTorch Speedup (Replaces xformers)
pipe.enable_attention_slicing()
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

print("🚀 Pipeline is live and GPU-accelerated.")

In [ ]:
# 1. Select a high-quality architectural sketch (Sample 150 is often cleaner)
sample_idx = 150
test_sketch = load_image(train_dataset[sample_idx]['conditioning_image'])

# 2. Professional Prompt Engineering
# Using specific IKEA-aesthetic keywords to trigger the fine-tuned weights
prompt = (
    "A professional IKEA catalogue photo, Scandinavian minimalist interior, "
    "light birch wood textures, soft natural daylight, high-end furniture, "
    "8k UHD, architectural photography, neutral color palette, realistic shadows"
)
negative_prompt = "blurry, low quality, distorted furniture, messy, dark, noisy"

# 3. High-Fidelity Generation
with torch.inference_mode():
    result = pipe(
        prompt,
        negative_prompt=negative_prompt,
        image=test_sketch,
        num_inference_steps=50, # Increased for smoother textures
        controlnet_conditioning_scale=0.8, # Lets AI add realistic depth
        guidance_scale=8.5 # Higher focus on the prompt
    ).images[0]

# 4. Final Comparison for Applicant A & B
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(test_sketch, cmap='gray'); ax[0].set_title("Input Structural Guide (Applicant A)")
ax[1].imshow(result); ax[1].set_title("Final Fine-Tuned Interior (Applicant B)")
for a in ax: a.axis('off')
plt.show()